In [43]:
pip install mlflow

In [45]:
pip install optuna


In [46]:
pip install dagshub

In [85]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow
import dagshub
import os
import optuna
import tqdm

In [86]:
# ✅ Set the correct Dagshub tracking URI (not GitHub!)
mlflow.set_tracking_uri("https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow")


# ✅ Set your Dagshub username and token (ensure token is kept secret)
os.environ["MLFLOW_TRACKING_USERNAME"] = "Mervin50"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "4f0ea46f344db4274bbaa99ff0d92ba94fcdc08b"



In [87]:
# Correct Dagshub MLflow URI
mlflow.set_tracking_uri("https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow")

# Set credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "Mervin50"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "4f0ea46f344db4274bbaa99ff0d92ba94fcdc08b"

# Initialize DagsHub MLflow integration
dagshub.init(repo_owner='Mervin50', repo_name='Uber-Demand-Prediction', mlflow=True)

# Set or create experiment
mlflow.set_experiment("model-selection")


Initialized MLflow to track repo "Mervin50/Uber-Demand-Prediction"

Repository Mervin50/Uber-Demand-Prediction initialized!

<Experiment: artifact_location='mlflow-artifacts:/b83d4cad915b4c1daf9d3a642caa1e17', creation_time=1746318342413, experiment_id='0', last_update_time=1746318342413, lifecycle_stage='active', name='model-selection', tags={}>

In [88]:
# load the training and test data

train_data_path = "/content/df_final_train.csv"
test_data_path = "/content/df_final_test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,10.0,10.0,10.0,13.0,0,10,10.0,4
2016-01-01 01:15:00,10.0,10.0,10.0,10.0,0,10,10.0,4
2016-01-01 01:30:00,10.0,10.0,10.0,10.0,0,10,10.0,4
2016-01-01 01:45:00,10.0,10.0,10.0,10.0,0,10,10.0,4
2016-01-01 02:00:00,10.0,10.0,10.0,10.0,0,10,10.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,10.0,10.0,10.0,10.0,29,10,10.0,0
2016-02-29 23:00:00,10.0,10.0,10.0,10.0,29,10,10.0,0
2016-02-29 23:15:00,10.0,10.0,10.0,10.0,29,10,10.0,0


In [89]:
# missing value in training data

train_df.isna().sum()

,0
lag_1,0
lag_2,0
lag_3,0
lag_4,0
region,0
total_pickups,0
avg_pickups,0
day_of_week,0


In [90]:
# missing values in the test data

test_df.isna().sum()

,0
lag_1,0
lag_2,0
lag_3,0
lag_4,0
region,0
total_pickups,0
avg_pickups,0
day_of_week,0


In [91]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [92]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,10.0,10.0,10.0,13.0,0,10.0,4
2016-01-01 01:15:00,10.0,10.0,10.0,10.0,0,10.0,4
2016-01-01 01:30:00,10.0,10.0,10.0,10.0,0,10.0,4
2016-01-01 01:45:00,10.0,10.0,10.0,10.0,0,10.0,4
2016-01-01 02:00:00,10.0,10.0,10.0,10.0,0,10.0,4


In [93]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [94]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,10.0,10.0,10.0,10.0,0,10.0,1
2016-03-01 00:15:00,11.0,10.0,10.0,10.0,0,10.0,1
2016-03-01 00:30:00,10.0,11.0,10.0,10.0,0,10.0,1
2016-03-01 00:45:00,10.0,10.0,11.0,10.0,0,10.0,1
2016-03-01 01:00:00,10.0,10.0,10.0,11.0,0,10.0,1


In [95]:
from sklearn import set_config

set_config(transform_output="pandas")

In [96]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [97]:
encoder

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['region', 'day_of_week'])])

In [98]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [99]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:

        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)

        if model_name == "LR":
            model = LinearRegression()

        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf,
                                          max_depth=max_depth_rf,
                                          random_state=42, n_jobs=-1)

        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb,
                                              learning_rate=learning_rate_gb,
                                             random_state=42)

        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)

        # log the model name
        mlflow.log_param("model_name",model_name)

        # log the model parameters
        mlflow.log_params(model.get_params())

        # fit on the data
        model.fit(X_train_encoded,y_train)

        # get the predictions
        y_pred = model.predict(X_test_encoded)

        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)

        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [100]:
# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)

    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)

[I 2025-05-04 00:26:53,471] A new study created in memory with name: model_selection
[I 2025-05-04 00:27:02,691] Trial 1 finished with value: 0.0010679158292697401 and parameters: {'model_name': 'RF', 'n_estimators_rf': 20, 'max_depth_rf': 9}. Best is trial 1 with value: 0.0010679158292697401.


🏃 View run valuable-fly-767 at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/ddb4e03d310643bd8d34637643ca6f68
🧪 View experiment at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-05-04 00:27:04,376] Trial 2 finished with value: 0.0022700453129822283 and parameters: {'model_name': 'LR'}. Best is trial 1 with value: 0.0010679158292697401.


🏃 View run zealous-gnat-194 at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/e42f54c6c0e74cafab12dad717c91aeb
🧪 View experiment at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-05-04 00:27:10,160] Trial 3 finished with value: 0.0014810825266393505 and parameters: {'model_name': 'RF', 'n_estimators_rf': 20, 'max_depth_rf': 6}. Best is trial 1 with value: 0.0010679158292697401.


...

🏃 View run redolent-colt-945 at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/0aedbefc948542f1be9326e4a1574213
🧪 View experiment at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0
🏃 View run skittish-newt-374 at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/7424b648d1d9432fa5957ffeca4fd531
🧪 View experiment at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-05-04 00:34:09,820] Trial 48 finished with value: 0.0022700453129822283 and parameters: {'model_name': 'LR'}. Best is trial 1 with value: 0.0010679158292697401.
[I 2025-05-04 00:34:18,630] Trial 46 finished with value: 0.0010732978260651466 and parameters: {'model_name': 'RF', 'n_estimators_rf': 60, 'max_depth_rf': 9}. Best is trial 1 with value: 0.0010679158292697401.


🏃 View run silent-sponge-957 at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/640b8e3f7cbc4a8296c530c502500804
🧪 View experiment at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0


[I 2025-05-04 00:34:26,332] Trial 49 finished with value: 0.0010930424696939899 and parameters: {'model_name': 'RF', 'n_estimators_rf': 60, 'max_depth_rf': 8}. Best is trial 1 with value: 0.0010679158292697401.


🏃 View run illustrious-deer-656 at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/d2a195b72aed413080b78eb987014973
🧪 View experiment at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0
🏃 View run best_model at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0/runs/26ac1a16b1424c06a0f22e54be776e89
🧪 View experiment at: https://dagshub.com/Mervin50/Uber-Demand-Prediction.mlflow/#/experiments/0


In [101]:
# best value

study.best_value

0.0010679158292697401

In [102]:
# best parameters

study.best_params

{'model_name': 'RF', 'n_estimators_rf': 20, 'max_depth_rf': 9}

In [103]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

,count
params_model_name,
RF,31
LR,7
XGBR,7
GBR,5


In [104]:
from optuna.visualization import (
    plot_optimization_history,
    plot_parallel_coordinate,
    plot_param_importances
)

In [105]:
plot_optimization_history(study)

In [106]:
plot_parallel_coordinate(study, params=["model_name"])

In [107]:
# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded)
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

The training error is  0.004087547467779396
The test error is  0.0022700453129822283


In [108]:
lr.coef_

array([ 1.07188818e-02,  2.04381318e-02,  1.58179478e-02,  1.22019589e-02,
       -1.69417797e-03,  9.40430213e-03,  1.76632645e-02,  2.23484703e-02,
        1.12982045e-02,  5.77093599e-03,  1.56360199e-02,  2.29339303e-02,
        2.24345394e-02, -7.49473355e-04,  8.61125591e-03,  2.17939256e-02,
        6.31277515e-03,  1.56799014e-03,  1.36345703e-02, -3.41022413e-03,
       -1.41208551e-02,  1.23408592e-02,  1.44708670e-02,  8.54521593e-03,
        1.56865637e-02,  8.99047121e-03,  7.25532432e-03,  2.02947358e-02,
       -5.40647528e-03,  9.76550792e-04,  9.76550792e-04,  9.31535919e-04,
       -4.95531112e-02, -4.15475346e-02,  3.82372428e-04, -5.56543756e-01,
       -3.68042454e-01, -1.95516216e-01, -2.02177869e-01,  2.43401886e+00])

In [109]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)

    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)

    # train the model
    ridge.fit(X_train_encoded, y_train)

    # get predictions
    y_pred = ridge.predict(X_test_encoded)

    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss


In [110]:
# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

[I 2025-05-04 00:35:44,441] A new study created in memory with name: tune_model


In [111]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-05-04 00:35:45,422] Trial 1 finished with value: 0.002246779177660719 and parameters: {'alpha': 88.28707879603172}. Best is trial 1 with value: 0.002246779177660719.
[I 2025-05-04 00:35:45,442] Trial 0 finished with value: 0.0022490878851674748 and parameters: {'alpha': 78.22171930856442}. Best is trial 1 with value: 0.002246779177660719.
[I 2025-05-04 00:35:46,053] Trial 3 finished with value: 0.002251792277468619 and parameters: {'alpha': 66.75318136679329}. Best is trial 1 with value: 0.002246779177660719.
[I 2025-05-04 00:35:46,059] Trial 2 finished with value: 0.002256509560063078 and parameters: {'alpha': 47.8412624314159}. Best is trial 1 with value: 0.002246779177660719.
[I 2025-05-04 00:35:46,599] Trial 5 finished with value: 0.0022589705499201414 and parameters: {'alpha': 38.44784977208617}. Best is trial 1 with value: 0.002246779177660719.
[I 2025-05-04 00:35:46,612] Trial 4 finished with value: 0.0022599992648744604 and parameters: {'alpha': 34.61134166110391}. Best

In [112]:
# best parameters

study.best_params

{'alpha': 99.99201573335183}

In [113]:
# best value

study.best_value

0.0022441624883497036